# Actividad Práctica Experimental 3


En esta práctica se evealuarán conceptos de análisis de datos y de regresión lineal múltiple.

## Librerías a utilizar

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns

## Preprocesamiento de los Datos


El dataset utilizado registra los datos sobre automóviles, con su precio estimado de ventas y demás estadísticas de cada automóvil.

> Nota: El dataset está disponible en Kaggle en el siguiente enlace [Car Dataset](https://www.kaggle.com/datasets/CooperUnion/cardataset)

In [ ]:
df = pd.read_csv("./car-dataset.csv")
df.head()

In [ ]:
df.info()

In [ ]:
df.shape

In [ ]:
df.describe()

### Limpieza de Datos

#### Revisión de valores nulos

In [ ]:
df.isna().sum()

Se eliminan las filas que contienen valores nulos en las columnas de Caballos de Fuerza del motor, Cilindraje del motor y Categoría del mercado del carro.

In [ ]:
normalized_df = df.dropna(subset=["Engine HP", "Engine Cylinders"])
normalized_df["Market Category"].fillna("Unknown", inplace=True)
normalized_df.isna().sum()

In [ ]:
normalized_df.shape

#### Revisión de duplicados

In [ ]:
normalized_df.duplicated().sum()

Se elimina el valor duplicado

In [ ]:
normalized_df = normalized_df.drop_duplicates()
normalized_df.duplicated().sum()

#### Eliminación de columnas


Se eliminan las columnas que a simple vista no aportan información significativa en la predicción de la variable objetivo.

In [ ]:
normalized_df.drop(labels=["Make", "Model", "Engine Fuel Type", "Driven_Wheels", "Number of Doors", "Vehicle Size"] , axis=1, inplace=True)
normalized_df.head()

Se extraen las etiquetas múltiples de la columna "Market Category" y se hace One Hot Encoding para estudiar sus etiquetas por separado.

In [ ]:
# Step 1: Split the multi-label entries
all_categories = set()
normalized_df['Market Category'].dropna().str.split(',').apply(all_categories.update)
all_categories = sorted(all_categories)

# Step 2: Create indicator columns
for category in all_categories:
    normalized_df[f'Market_Category_{category.strip()}'] = normalized_df['Market Category'].fillna('').str.contains(category, na=False).astype(int)

# Step 3: Drop original column
normalized_df.drop('Market Category', axis=1, inplace=True)

normalized_df.head()

#### Convertir las Columnas Categóricas a Numéricas


Se hace One Hot Encoding con las columnas de 'Transmission Type' y 'Vehicle Style'.

In [ ]:
df_encoded = pd.get_dummies(normalized_df, columns=['Transmission Type', 'Vehicle Style'], drop_first=True, dtype=int)
df_encoded.info()

#### Eliminación de Outliers


Los outliers presentes en el dataset pueden alterar la predicción, por lo cual se eliminan usando el método de eliminación de distribución estándar.

In [ ]:
def remove_outliers_std(df, column, threshold=3):
    mean = df[column].mean()
    std = df[column].std()
    df_filtered = df[(df[column] > mean - threshold*std) & 
                    (df[column] < mean + threshold*std)]
    return df_filtered


df_clean = remove_outliers_std(df_encoded, 'MSRP', threshold=3)

## Visualización de los datos

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
correlation_matrix = df_clean.corr()
plt.figure(figsize=(30, 26))
sns.heatmap(correlation_matrix, annot=True, fmt=".2f", cmap="coolwarm", square=True, cbar_kws={"shrink": .8})
plt.title("Matriz de Correlación")
plt.show()

En este caso, se puede observar que hay algunas variables que se correlacionan con la varible objetivo (MSRP) como lo son las variables de Caballos de fuerza del motor, el cilindraje del motor (Engine Cylinders), la categoría de mercado exótica (Market_Category_Exotic), de alto rendimiento (Market_Category_High-Performance) y el año.

In [ ]:
sns.pairplot(df_clean[["Year", "Engine Cylinders", "Engine HP", "Market_Category_Exotic", "Market_Category_High-Performance", "MSRP"]])

### Variables a Elegir


Luego de visualizar la correlación de las variables, se tomaron en cuenta las siguientes variables para la construcción del conjunto de entrenamiento y de prueba.



Se toman en cuenta las variables de Caballos de fuerza del motor, la categoría de mercado exótica, de alto rendimiento y el año.



La variable del cilidraje del motor tiene una buena correlación con la variable objetivo, pero debido a la colinearidad alta con la variable de
caballos de fuerza del motor, solo se elige una.

In [ ]:
x = df_clean[["Year", "Engine HP", "Market_Category_Exotic", "Market_Category_High-Performance"]]
y = df_clean[["MSRP"]]

## Separación de Conjunto de Entrenamiento y Prueba


Ahora se hace la separación de los datos de entrenamiento y de prueba.

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

En este caso se toma un tamaño del conjunto de prueba del 20% con una semilla aleatoria de 50.

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.20, random_state=50)

## Configuración del algoritmo

Se inicializa el modelo

In [ ]:
model = LinearRegression()

## Entrenamiento del modelo


Se entrena el modelo con los datos del conjunto de entrenamiento.

In [ ]:
model.fit(x_train, y_train)

## Predicción


Se toman las predicciones hechas por el modelo utilizando los datos de prueba.

In [ ]:
y_prediction = model.predict(x_test)
y_prediction

## Pruebas del modelo


Se evalúa el modelo utilizando métricas como lo es el mean absolute error y el r2 score.

In [ ]:
from sklearn.metrics import mean_absolute_error, r2_score

### Intercepto y coeficientes de la regresión

In [ ]:
print(f"Intercepto del eje Y: {model.intercept_}")
print(f"Coeficientes: {model.coef_}")

### Error Absoluto Medio y Coeficiente de Determinación $R^2$

In [ ]:
print(f"Error Absoluto Medio: {mean_absolute_error(y_true=y_test, y_pred=y_prediction)}")
print(f"Error Cuadrático Medio: {r2_score(y_true=y_test, y_pred=y_prediction)}")

### Valores Reales vs Valores Previstos


Se revisan las predicciones del modelo vs los valores reales del modelo para ver la precisión del modelo.

In [ ]:
plt.figure(figsize=(8, 6))
plt.scatter(y_test, y_prediction, color='blue', label='Predicciones')
plt.plot([y.min(), y.max()], [y.min(), y.max()], color='red', linestyle='--', label='Predicción Perfecta')
plt.xlabel('Valores Reales')
plt.ylabel('Valores Previstos')
plt.title('Valores Reales vs Previstos')
plt.legend()
plt.grid()
plt.show()

En este gráfico se observa que hay una gran parte de las predicciones que se acercan a los valores reales, a excepción de algunos casos que
la predicción se aleja de la recta, esto se debe a que faltan más indicadores para determinar con mayor precisión el precio estimado del manufacturero (MSRP).

## Preguntas de Control


1. ### ¿Qué indican los signos y magnitudes de los coeficientes del dataset?


Los coeficientes son aquellos que establecen relación entre las variables independientes ($X_{n}$) y la variable dependiente ($Y$).

La magnitud de los coeficientes indica en que medida aporta cada variable a la variable objetivo, por ejemplo, mientras mayor sea la magnitud absoluta de una variable, esta influencia de mayor manera la variable objetivo.

Los signos de los coeficientes indica la relación que tiene la variable predictora con respecto a la variable objetivo, por ejemplo, si el coeficiente de una variable es positivo la relación entre esa variable y la variable objetivo es **directamente proporcional** (si la variable predictora aumenta, la variable objetivo también aumenta), y si el coeficiente es negativo, la relación entre la variable predictora y la variable objetivo es **inversamente proporcional** (si la variable predictora aumenta, la variable objetivo disminuye).

2. ### ¿Qué variable parece tener mayor impacto?


En este caso la variable que parece tener mayor impacto con la variable objetivo es la correspondiente a los caballos de fuerza del motor (Engine HP), debido a que la correlación con esta variable y la variable objetivo es alta.

3. ### Crea un modelo solo con la variable TV. Compara su rendimiento con el modelo múltiple.

En este caso se utilizará la variable de Engine HP para ver su rendimiento con respecto al rendimiento del modelo de regresión múltiple.

In [ ]:
simple_model = LinearRegression()

In [ ]:
x = df_clean[["Engine HP"]]
y = df_clean[["MSRP"]]

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.20, random_state=50)

In [ ]:
simple_model.fit(x_train, y_train)

In [ ]:
y_prediction = simple_model.predict(x_test)
y_prediction

In [ ]:
print(f"Intercepto del eje Y: {simple_model.intercept_}")
print(f"Coeficientes: {simple_model.coef_}")

In [ ]:
print(f"Error Absoluto Medio: {mean_absolute_error(y_true=y_test, y_pred=y_prediction)}")
print(f"Error Cuadrático Medio: {r2_score(y_true=y_test, y_pred=y_prediction)}")

In [ ]:
plt.figure(figsize=(8, 6))
plt.scatter(y_test, y_prediction, color='blue', label='Predicciones')
plt.plot([y.min(), y.max()], [y.min(), y.max()], color='red', linestyle='--', label='Predicción Perfecta')
plt.xlabel('Valores Reales')
plt.ylabel('Valores Previstos')
plt.title('Valores Reales vs Previstos')
plt.legend()
plt.grid()
plt.show()

En este modelo se puede encontrar una diferencia en el gráfico de las predicciones, los valores de la predicción se alejan aún más de la recta de los valores reales, además las métricas de evaluación se ve reflejado la pérdida de precisión del modelo, reduciendo la determinación $R^2$ y aumentando el error absoluto medio. Esto se debe a que la variable objetivo depende de múltiples factores para cambiar, una sola variable no puede representar en por completo la imfluencia sobre el precio del automóvil.